# Radius Projection

This notebook is a successor to the one made for predicting the eye of the storm and the radius retrieved from the model.
We perform data transformations to do two major tasks.
1. Retrieve the zipcodes that fall within the range of the storm.
2. Visualize the zipcodes on the map for verification.
3. Retrieve the housing prices and perform a join on the zipcode dataset.
4. Perform aggregations to retrieve the financial losses.

In [1]:
DATASET_LOC = "../data/uszips.csv"

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(DATASET_LOC)

In [4]:
df.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16834.0,100.9,72001,Adjuntas,"{""72001"": 98.74, ""72141"": 1.26}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37642.0,479.2,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,49075.0,551.7,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,5590.0,48.7,72093,Maricao,"{""72093"": 82.27, ""72153"": 11.66, ""72121"": 6.06}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,25542.0,265.7,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.82, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico


In [9]:
# Alternative approach

import math

# Haversine formula to calculate great-circle distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers. Use 3959 for miles.
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    
    a = math.sin(delta_phi / 2) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    return R * c

# Example: Define center and radius
# TODO: This below portion needs to be retrieved from the model.
center_lat, center_lon = 28.5383, -81.3792  # Orlando, FL
radius_km = 50  # 50 km

# Filter ZIP codes within the radius
df['distance'] = df.apply(
    lambda row: haversine(center_lat, center_lon, row['lat'], row['lng']), axis=1
)
zipcodes_in_radius = df[df['distance'] <= radius_km]

# Output results
print(zipcodes_in_radius[['zip', 'lat', 'lng', 'distance']])

         zip       lat       lng   distance
10775  32701  28.66509 -81.36916  14.132435
10777  32703  28.65579 -81.52139  19.062680
10778  32707  28.66255 -81.31403  15.210535
10779  32708  28.68751 -81.27086  19.675324
10780  32709  28.49266 -80.98801  38.556906
...      ...       ...       ...        ...
11460  34771  28.26647 -81.11908  39.508626
11461  34772  28.16534 -81.27273  42.759988
11464  34786  28.48161 -81.55501  18.298573
11465  34787  28.48309 -81.62833  25.104701
11467  34797  28.73517 -81.81813  48.105897

[93 rows x 4 columns]


After retrieving the zipcodes that fall into the range/radius of the storm. We can use `folium` to display the zipcodes on a map.
Interesting note: places without any houses near them are not allocated zipcodes. Zipcodes are meant to facilitate mail delivery and as such exist in inhabited areas.

In [12]:
import folium
import pandas as pd

# Create a map centered on the specified latitude and longitude
m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add markers for each ZIP code
for _, row in zipcodes_in_radius.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=f"ZIP: {row['zip']}, Distance: {row['distance']} km",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# Save the map to an HTML file
m.save("map_with_zipcodes.html")

# Display the map directly in Jupyter Notebook (if using one)
m